In [1]:
%config IPCompleter.greedy=True

In [5]:
from PIL import Image
import numpy as np
import io
import os
import torch
import pickle

In [62]:
#Bilder schwarz weiß machen optional
directory = 'F:\\image_classifier\\images_with_pen\\resized'

for filename in os.listdir(directory):
    img = Image.open(directory+"\\"+filename)
    new_img =img.convert("L")
    new_img.save(directory+"\\"+filename)

OSError: cannot identify image file 'F:\\image_classifier\\images_with_pen\\resized\\BulkResizePhotos.html'

In [63]:

directory = 'F:\\image_classifier\\images_without_pen\\resized'

for filename in os.listdir(directory):
    img = Image.open(directory+"\\"+filename)
    new_img =img.convert("L")
    new_img.save(directory+"\\"+filename)

In [39]:
#Anzahl der Bilder
directory = 'F:\\image_classifier\\images_without_pen\\resized'
print(len(os.listdir(directory)))
directory = 'F:\\image_classifier\\images_with_pen\\resized'
print(len(os.listdir(directory)))

4171
5502


In [40]:
4170+5501

9671

In [41]:
imageListWithout =[torch.zeros(1)]*4170

In [44]:
#Bilder ohne Stift array convertieren
directory = 'F:\\image_classifier\\images_without_pen\\resized'
i=0
for filename in os.listdir(directory):
    img = Image.open(directory+"\\"+filename)
    arr=array(img)
    imageListWithout[i]=torch.from_numpy(arr)
    i+=1

In [45]:
imageListWith =[torch.zeros(1)]*5501

In [47]:
#Bilder mit Stift in array convertieren
directory = 'F:\\image_classifier\\images_with_pen\\resized'
i=0
for filename in os.listdir(directory):
    img = Image.open(directory+"\\"+filename)
    arr=array(img)
    imageListWith[i]=torch.from_numpy(arr)
    i+=1

In [48]:
sample = {'image': imageListWith[0], 'label': 1.0}

In [49]:
imageList =[sample]*9671

In [50]:
len(imageList)

9671

In [51]:
#Alles zusammenpacken
imageList=[torch.zeros(1)]*9671
for i in range(4170):
    imageList[i]={'image' : imageListWithout[i]/255.0, 'label' : 0.0}
for i in range(5501):
    imageList[i+4170]={'image' : imageListWith[i]/255.0, 'label' : 1.0}


In [52]:
#Durchmischen
import random
random.shuffle(imageList)

In [53]:
#Test und trainings Daten aufteilen
split_size = int(len(imageList)*0.7)

trainData, testData = imageList[:split_size], imageList[split_size:]

In [69]:
print(len(trainData))
print(len(testData))

6769
2902


In [74]:
#Alles speichern
dataSet = [trainData,testData]
with open('F:\\image_classifier\\file.txt', "wb") as f:
    pickle.dump(dataSet, f, pickle.HIGHEST_PROTOCOL)

^^ Data Preparation ^^

˅˅      Training    ˅˅

In [1]:
import torch
import torchvision
import pickle
import os
import random
import torch.nn as nn4
import torch.nn.functional as F

In [2]:
class trainingDataLoader(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        self.trainData = pickle.load( open("F:\\image_classifier\\file.txt", "rb"))[0]
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        return self.trainData[index]
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return len(self.trainData)

In [3]:
class testDataLoader(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        self.testData = pickle.load( open("F:\\image_classifier\\file.txt", "rb"))[1]
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        return self.testData[index]
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return len(self.testData)

In [4]:
train_dataset = trainingDataLoader()
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)
test_dataset = testDataLoader()
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [16]:
#Netzwerk
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.batchnorm = nn.BatchNorm2d(1)
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(64*9*9, 5000)
        self.fc2 = nn.Linear(5000, 1)
    def forward(self, x):
        #out = self.batchnorm(x)
        out = self.layer1(x)
        out = self.layer2(out)
        #print(out.shape)
        out = out.reshape(out.size(0), -1)
        #out = self.Batchnorm(out)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
net=ConvNet()

In [17]:
import torch.optim as optim

criterion = nn.MSELoss()
#optimizer = optim.SDG(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [18]:
#Training
total_step = len(train_loader)
lastaccuracy =0;
acc_list = []
for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs=data['image']
        labels = data['label']
        inputs = inputs.type('torch.FloatTensor')
        inputs= torch.reshape(inputs,[len(inputs),3,150,150])
        labels = labels.type('torch.FloatTensor')
        labels= torch.reshape(labels,[len(labels),1])
        # zero the parameter gradients
        optimizer.zero_grad()
        net.eval
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # calculate Accuracy
        correct=0
        predicted = torch.round(outputs)
        total = labels.size(0)
        #correct += (predicted == labels).sum().item()
        for prediction, label in zip(predicted,labels):
            if(label.item() == prediction.item()):
                correct+=1 
        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
           print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, 3, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
        
        #running_loss = 0.0

print('Finished Training')

Epoch [1/3], Step [20/106], Loss: 0.5357, Accuracy: 48.44%
Epoch [1/3], Step [40/106], Loss: 0.4876, Accuracy: 45.31%
Epoch [1/3], Step [60/106], Loss: 0.4371, Accuracy: 42.19%
Epoch [1/3], Step [80/106], Loss: 0.3458, Accuracy: 46.88%
Epoch [1/3], Step [100/106], Loss: 0.2908, Accuracy: 50.00%
Epoch [2/3], Step [20/106], Loss: 0.2803, Accuracy: 40.62%
Epoch [2/3], Step [40/106], Loss: 0.2544, Accuracy: 46.88%
Epoch [2/3], Step [60/106], Loss: 0.2505, Accuracy: 46.88%
Epoch [2/3], Step [80/106], Loss: 0.2490, Accuracy: 53.12%
Epoch [2/3], Step [100/106], Loss: 0.2478, Accuracy: 54.69%
Epoch [3/3], Step [20/106], Loss: 0.2531, Accuracy: 51.56%
Epoch [3/3], Step [40/106], Loss: 0.2488, Accuracy: 54.69%
Epoch [3/3], Step [60/106], Loss: 0.2615, Accuracy: 46.88%
Epoch [3/3], Step [80/106], Loss: 0.2531, Accuracy: 51.56%
Epoch [3/3], Step [100/106], Loss: 0.2569, Accuracy: 48.44%
Finished Training


In [20]:
#GEnauigkeit ermitteln
correct = 0
total = 0
net.eval()
with torch.no_grad():
    for data in test_loader:
        images=data['image']
        labels = data['label']
        images = images.type('torch.FloatTensor')
        images= torch.reshape(images,[len(images),3,150,150])
        labels = labels.type('torch.FloatTensor')
        predicted = torch.round(outputs)
        total += labels.size(0)
        #correct += (predicted == labels).sum().item()
        for prediction, label in zip(predicted,labels):
            if(label.item() == prediction.item()):
                correct+=1

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 42 %


In [49]:
#Richtiges speichern
torch.save(net.state_dict,'F:\\image_classifier\\model_params.dat')

D:\python\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type ConvNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [50]:
#falsches speichern (wird hier benutzt)
torch.save(net,'F:\\image_classifier\\model_params.dat')

D:\python\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type ConvNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [51]:
#Laden vom Netzwerk
testnet = ConvNet()
testnet =torch.load('F:\\image_classifier\\model_params.dat')